<!-- SPDX-License-Identifier: CC-BY-4.0 -->
<!-- Copyright Contributors to the ODPi Egeria project 2024. -->

![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Workbook

# Loading content packs

## Introduction

A content pack is a collection of curated metadata that can be loaded into your metadata store.  Egeria includes a number of content packs that can be used to explore various features of egeria ot to test your integration code.  This workbook shows how to load a content pack that is stored as a file.  These files have a file extension of `omarchive`.

## Listing the content packs

A list of content packs installed in Egeria's runtime container can be retrieved using the `hey_egeria_cat show list-archives` command or through the `EgeriaTech` python libraries:

----

In [10]:
import os
view_server = os.environ.get("VIEW_SERVER","view-server")
url = os.environ.get("EGERIA_VIEW_SERVER_URL","https://host.docker.internal:9443")
user_id = os.environ.get("EGERIA_USER", "peterprofile")
user_pwd = os.environ.get("EGERIA_USER_PASSWORD")

from pyegeria import EgeriaTech
import asyncio
import nest_asyncio
nest_asyncio.apply()

from datetime import datetime

egeria_tech = EgeriaTech(view_server, url, user_id, user_pwd)
egeria_tech_token = egeria_tech.create_egeria_bearer_token()

In [11]:

archiveList = egeria_tech.get_elements_by_property_value("omarchive", ["fileExtension"], "ArchiveFile")
if archiveList:
    print("Known Archive Files:")
    for archiveFile in archiveList:
        if archiveFile:
            properties = archiveFile.get('properties')
            if properties:
               print("* " + properties.get('pathName'))
            

Known Archive Files:
* /deployments/content-packs/SimpleAPICatalog.omarchive
* /deployments/content-packs/SimpleCatalog.omarchive
* /deployments/content-packs/SimpleEventCatalog.omarchive
* /deployments/content-packs/CocoGovernanceEngineDefinitionsArchive.omarchive
* /deployments/content-packs/CocoClinicalTrialsTemplatesArchive.omarchive
* /deployments/content-packs/CocoComboArchive.omarchive
* /deployments/content-packs/CocoGovernanceEngineDefinitionsArchive.omarchive
* /deployments/content-packs/OpenMetadataTypes.omarchive
* /deployments/content-packs/CocoGovernanceProgramArchive.omarchive
* /deployments/content-packs/SimpleDataCatalog.omarchive
* /deployments/content-packs/CocoTypesArchive.omarchive
* /deployments/content-packs/CocoOrganizationArchive.omarchive
* /deployments/content-packs/CocoSustainabilityArchive.omarchive
* /deployments/content-packs/CloudInformationModel.omarchive
* /deployments/content-packs/SimpleGovernanceCatalog.omarchive
* /deployments/content-packs/CoreCon

----

A description of these content packs can be found in the [Egeria documentation](https://egeria-project.org/content-packs/).


To load one of these archives, you need to choose a metadata access server where you want the metadata to be stored and then call the load function with the name of the archive.

The code below lists the metadata access servers currently known to Egeria:

----

In [12]:
serverList = egeria_tech.get_technology_type_elements("Metadata Access Server")

if serverList:
    print("Known Metadata Access Servers:")
    for server in serverList:
        if server:
            guid="???"
            serverName="????"
            description="???????"
            elementHeader = server.get('elementHeader')
            if elementHeader:
                guid = elementHeader.get('guid')
            properties = server.get('referenceableProperties')
            if properties:
                extendedProperties = properties.get('extendedProperties')
                if extendedProperties:
                    serverName=extendedProperties.get('resourceName')
                    description=extendedProperties.get('description')
            print("* " + serverName + " (" + guid + ") - " + description)
            

Known Metadata Access Servers:
* simple-metadata-store (7a28acc2-a0b6-45a9-973d-2718fae3a87e) - A metadata store that supports Open Metadata Access Services (OMASs) without event notifications.
* active-metadata-store (828a1280-9251-4509-b7d4-22f326690cec) - A metadata store that supports Open Metadata Access Services (OMASs) with event notifications.  It provides metadata to view-server, engine-host and integration-daemon.


----

In the code below, the */deployments/content-packs/CocoComboArchive.omarchive* archive is loaded into the *active-metadata-server*.  It does not matter if the same archive is loaded multiple times.  This does not result in duplicate versons of the same element.

-----

In [13]:

def get_server_guid(serverName):
    serverDetails = egeria_tech.get_elements_by_property_value(serverName, ["resourceName"], "SoftwareServer")
    if serverDetails:
        elementHeader = serverDetails[0].get('elementHeader')
        if elementHeader:
            return elementHeader.get('guid')
    return "???"
            

serverGUID = get_server_guid("active-metadata-store")

egeria_tech.add_archive_file("/deployments/content-packs/CocoComboArchive.omarchive", serverGUID)


----

There is no response, however the archive is loaded if it responds without an error.  If an error comes back indicating that there has been a time out, then the archive will continue to load in the server until it is complete.

----